recommendation sys


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
brand_data = pd.read_csv("brand_dataset.csv")
influencer_data = pd.read_csv("influencer_dataset.csv")

In [ ]:
brand_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location
0,1,Brand72,Nutrition Products,15-24,Female,Kerala
1,2,Brand88,Smartphones,All,Male,Gujarat
2,3,Brand55,Home Appliances,18-30,Male,Kerala
3,4,Brand84,Gadgets,15-24,Male,Kerala
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu


In [ ]:
brand_data['brand_name'].unique

<bound method Series.unique of 0      Brand72
1      Brand88
2      Brand55
3      Brand84
4      Brand43
        ...   
595     Brand8
596    Brand80
597    Brand29
598     Brand7
599    Brand89
Name: brand_name, Length: 600, dtype: object>

In [ ]:
influencer_data.head()

,id,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


In [ ]:
brand_data.isnull().sum()

id                  0
brand_name          0
product_category    0
target_age          0
target_gender       0
target_location     0
dtype: int64

In [ ]:

influencer_data.isnull().sum()

id                   0
Influencer_name      0
Handle               0
Followers            0
Posts                0
Likes                0
Category             0
Sentiment            0
audience_location    0
audience_age         0
audience_Gender      0
Platform             0
Engagement_rate      0
dtype: int64

merging both tables

In [ ]:
merged_data = pd.merge(brand_data, influencer_data, on = 'id', how = 'right')

In [ ]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


Based on Search (using Category)

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stem = [stemmer.stem(w) for w in tokens]
    return " ".join(stem)


In [ ]:
merged_data['stemmed_tokens'] =merged_data.apply(lambda row: tokenize_stem(row['Category']+ ' ' + row['audience_location']+ ' ' + row['audience_age']+ ' ' + row['Platform']), axis=1)

In [ ]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate,stemmed_tokens
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302,food kerala 15-24 tiktok
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254,fashion gujarat all twitter
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283,food kerala 18-30 instagram
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293,lifestyl kerala 15-24 facebook
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749,lifestyl tamil nadu 40-50 instagram


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(tokenizer=tokenize_stem)

def cosine_sim(txt1,txt2):
    tfid_matrix = tfidvectorizer.fit_transform([txt1,txt2])
    return cosine_similarity(tfid_matrix)[0][1]

In [ ]:
def search_product(query):
    stemmed_query = tokenize_stem(query)
    #calcualting cosine similarity between query and stemmed tokens columns
    merged_data['similarity'] = merged_data['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res = merged_data.sort_values(by=['similarity'],ascending=False).head(10)[['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]
    return res

In [ ]:
# search based on category, audience_location, age, gender

search_product('Food').sort_values(by=['Engagement_rate'],ascending=False)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
477,John Smith,Food,@isabella.m,574804,Goa,18-25,Female,TikTok,6.873995
177,Sophia Wilson,Food,@mia.lee,824940,Punjab,18-30,All,Facebook,5.854850
301,Mia Lee,Food,@noah.d,689505,Bihar,25-40,Female,Facebook,5.810110
443,John Smith,Food,@noah.d,574454,Kerala,18-25,All,Facebook,4.938428
180,Isabella Moore,Food,@noah.d,886367,Delhi,25-40,Male,Facebook,4.644126
109,John Smith,Food,@john.smith,949199,Kerala,18-28,Male,Facebook,2.703437
381,Jane Doe,Food,@sophia.w,780380,Goa,25-40,Female,YouTube,2.381148
330,Noah Davis,Food,@sophia.w,416450,Haryana,30-45,All,Facebook,2.174571
569,Mia Lee,Food,@liam.brown,709360,India,25-34,Female,YouTube,1.596791
163,Mia Lee,Food,@mia.lee,840086,Madhya Pradesh,30-45,Female,Facebook,1.384382


Collaborative Recommendation -- Test

In [ ]:
user_item_matrix = merged_data.pivot_table(index='id', columns='Influencer_name', values='Engagement_rate',aggfunc='mean').fillna(0).astype(int)

In [ ]:
user_similarity = cosine_similarity(user_item_matrix)

In [ ]:
target_brand_id = 4
target_user_index = user_item_matrix.index.get_loc(target_brand_id)

In [ ]:
user_similarities = user_similarity[target_user_index]

similar_user_indices = user_similarities.argsort()[::-1][1:]

In [ ]:
recommend_items = []

for user_index in similar_user_indices:
    rated_by_similar_user = user_item_matrix.iloc[user_index]
    not_rated_by_target_user = (rated_by_similar_user==0) & (user_item_matrix.iloc[target_user_index]==0)

    recommend_items.extend(user_item_matrix.columns[not_rated_by_target_user][:10])

recommended_items_details = merged_data[merged_data['Influencer_name'].isin(recommend_items)][['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]

In [ ]:
recommended_items_details.head(10)

,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,Jane Doe,Food,@emma.t,340770,Kerala,15-24,Male,TikTok,9.269302
1,Mia Lee,Fashion,@jane.doe,702603,Gujarat,All,All,Twitter,7.077254
2,John Smith,Food,@noah.d,113855,Kerala,18-30,Male,Instagram,36.526283
4,Mason White,Lifestyle,@john.smith,495171,Tamil Nadu,40-50,Female,Instagram,6.165749
5,Mia Lee,Tech,@noah.d,341829,Madhya Pradesh,40-50,All,TikTok,7.681326
6,Isabella Moore,Travel,@liam.brown,487464,Gujarat,30-45,All,Facebook,3.488463
7,John Smith,Music,@sophia.w,733223,Tamil Nadu,18-30,All,TikTok,5.502692
9,Isabella Moore,Health,@isabella.m,409843,Karnataka,22-40,Female,Twitter,10.080201
11,Mason White,Lifestyle,@john.smith,767339,Karnataka,18-25,Male,Twitter,5.214253
12,Isabella Moore,Gaming,@noah.d,885659,Kerala,30-45,Female,Facebook,0.620329
